In [1]:
import sys
import os

SCRIPT_DIR = os.path.dirname(os.path.abspath("..."))
sys.path.append(os.path.dirname(SCRIPT_DIR))
from training.generate import (generate_response, load_model_tokenizer_for_generate, 
                               get_special_token_id, get_special_token_id)
from training.consts import END_KEY, PROMPT_FORMAT, RESPONSE_KEY_NL
from training.trainer import PROMPT_FORMAT, create_data_set_from_json_list
from datasets import Dataset, DatasetDict, load_dataset, load_from_disk


/opt/bo_ling/python39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [78]:
file_to_instruction_map = {
    "/opt/home/bo_ling/dataset/michelangelo_so_long.jsonl": "Please answer the following MA helpdesk questions:",
    "/opt/home/bo_ling/dataset/docstrans.jsonl": "Please finish the following doc translation tasks:",
    "/opt/home/bo_ling/dataset/eats_goldenset.jsonl": "Please answer Uber eats products relation questions:"
}
json_file = "/opt/home/bo_ling/dataset/eats_goldenset.jsonl"
dataset_file = json_file.replace(".jsonl", ".hf")
dataset = create_data_set_from_json_list(json_file,
                                         file_to_instruction_map=file_to_instruction_map) 
dataset = dataset.train_test_split(test_size=0.1)
#dataset.save_to_disk(dataset_file)

Dataset generator downloaded and prepared to /home/prodadmin/.cache/huggingface/datasets/generator/default-71d75ecb741b2dcb/0.0.0. Subsequent calls will reuse this data.


In [14]:
new_dataset = load_from_disk("/opt/home/bo_ling/dataset/eats_goldenset.hf")
new_dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 157792
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 17533
    })
})

In [71]:
data = []
max_question_length=500
max_answer_length=500
with open("/opt/home/bo_ling/dataset/doc_transcript_pii_data.json") as json_file:
    json_data = json.load(json_file)
for d in json_data:
    try:
        instruction = d['instruction']+":"
        input_text = d["input"][:max_question_length]
        output_text = "\n" + d["output"][:max_answer_length]
        text = PROMPT_FORMAT.format(instruction=instruction,input_text=input_text,output_text=output_text)
        data.append({
                "instruction": instruction,
                "input": input_text, 
                "output": output_text,
                "text": text
            })
    except:
        pass
def gen():
    for d in data:
        yield d
dataset = Dataset.from_generator(gen)
dataset = dataset.train_test_split(test_size=0.1)
#dataset.save_to_disk("/opt/home/bo_ling/dataset/doc_transcript_pii_data.hf")

Dataset generator downloaded and prepared to /home/prodadmin/.cache/huggingface/datasets/generator/default-2da5259a7e982007/0.0.0. Subsequent calls will reuse this data.


In [2]:
new_dataset = load_from_disk("/opt/home/bo_ling/dataset/doc_transcript_pii_data.hf")
new_dataset

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 629797
    })
    test: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 69978
    })
})

In [18]:
parquet_file = "/opt/home/bo_ling/dataset/modeling_data_v1.parquet"
import pandas as pd
df = pd.read_parquet(parquet_file, engine='pyarrow')
count = 0

def gen():
    for index, row in df.iterrows():
        yield {"text": row['context_and_msg']}
dataset = Dataset.from_generator(gen)
dataset = dataset.train_test_split(test_size=0.1)
    
dataset.save_to_disk("/opt/home/bo_ling/dataset/modeling_data_v1.hf")



Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 1796 examples [00:00, 17886.03 examples/s]

Generating train split: 4053 examples [00:00, 20631.90 examples/s]

Generating train split: 6371 examples [00:00, 21793.42 examples/s]

Generating train split: 8686 examples [00:00, 22327.86 examples/s]

Generating train split: 11000 examples [00:00, 22468.39 examples/s]

Generating train split: 13360 examples [00:00, 22846.81 examples/s]

Generating train split: 15714 examples [00:00, 23069.83 examples/s]

Generating train split: 19192 examples [00:00, 23077.28 examples/s]

Generating train split: 21533 examples [00:00, 23166.52 examples/s]

Generating train split: 23885 examples [00:01, 23264.08 examples/s]

Generating train split: 27380 examples [00:01, 23230.48 examples/s]

Generating train split: 29736 examples [00:01, 23316.73 examples/s]

Generating train split: 33173 examples [00:01, 23167.25 examples/s]

Generating train split: 35542 examples [00:01, 

Dataset generator downloaded and prepared to /home/prodadmin/.cache/huggingface/datasets/generator/default-7895c5e842d084b9/0.0.0. Subsequent calls will reuse this data.




Saving the dataset (0/1 shards):   0%|                                                                                                                                                                                      | 0/152615 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):  12%|███████████████████▌                                                                                                                                                  | 18000/152615 [00:00<00:00, 173411.10 examples/s]

Saving the dataset (0/1 shards):  26%|███████████████████████████████████████████▌                                                                                                                          | 40000/152615 [00:00<00:00, 195499.43 examples/s]

Saving the dataset (0/1 shards):  41%|███████████████████████████████████████████████████████████████████▍                                                                                                  | 62000/152615 [00:00<00:0

In [23]:
new_dataset = load_from_disk("/opt/home/bo_ling/dataset/modeling_data_v1.hf")
new_dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 152615
    })
    test: Dataset({
        features: ['text'],
        num_rows: 16958
    })
})

In [24]:
new_dataset['train'][0]

{'text': "Specific Information: The restaurant name is Insomnia Cookies (164 Orchard St) and the order value is $19.93. The estimated delivery time is 01:45 and latest arrival time is 02:05. Order status is 6.\n\n###\n\nagent: Hi Charlotte, welcome to the chat support team. I'll be assisting you today. <|endofsentence|>\nagent: May I know your concern please? <|endofsentence|>\nagent: Charlotte, Are we still connected? <|endofsentence|>\nagent: I'm grateful that I was able to help you out today, Charlotte. As I have not heard back from you in a while, I am closing the chat. Please feel free to contact us again for any issues. Thank you for chatting with us today. Stay safe, stay healthy. <|endofsentence|>\nagent: Yes No <|endofsentence|>\nagent: Have we resolved this issue? <|endofsentence|>\n"}

In [7]:
import json 
data = []
max_question_length=500
max_answer_length=500
with open("/opt/home/bo_ling/dataset/gds_v4.json") as json_file:
    json_data = json.load(json_file)
for d in json_data:
    try:
        instruction = d['instruction'].strip().replace(":", " input:")
        input_text = d["ocr_text"][:max_question_length]
        output_text = "\n" + d["output"][:max_answer_length]
        text = PROMPT_FORMAT.format(instruction=instruction,input_text=input_text,output_text=output_text)
        data.append({
                "instruction": instruction,
                "input": input_text, 
                "output": output_text,
                "text": text
            })
    except:
        pass
def gen():
    for d in data:
        yield d
dataset = Dataset.from_generator(gen)
dataset.save_to_disk("/opt/home/bo_ling/dataset/gds_v4.hf")

Found cached dataset generator (/home/prodadmin/.cache/huggingface/datasets/generator/default-71e7c02640b57610/0.0.0)


In [8]:
json_data[10]

{'uuid': 'c65024cc-26bc-490a-91d2-62f840fc0f8b',
 'field': 'address_line1',
 'instruction': 'Extract address from the following: ',
 'ocr_text': 'California DRIVER LICENSE 081202 LN SOTO MUNGUIA FN ALEC PIERRE 9939 VAN RUITEN ST BELLFLOWER, CA 90706 SEX M AM DONOR USA DLY7679862 EXP 08/12/2025 DOB 08/12/2002 AGE 2 IN 7023 DD 08/04/2021606A3/E5FD/25 CLASS C END NONE RSTR NONE 08122002 HAIR BRN EYES HZL HGT 5\'-11" WGT 205 lb FEDERAL LIMITS APPLY ISS 10/14/2021',
 'output': '9939 VAN RUITEN ST'}

In [9]:
dataset[10]

{'instruction': 'Extract address from the following input:',
 'input': 'California DRIVER LICENSE 081202 LN SOTO MUNGUIA FN ALEC PIERRE 9939 VAN RUITEN ST BELLFLOWER, CA 90706 SEX M AM DONOR USA DLY7679862 EXP 08/12/2025 DOB 08/12/2002 AGE 2 IN 7023 DD 08/04/2021606A3/E5FD/25 CLASS C END NONE RSTR NONE 08122002 HAIR BRN EYES HZL HGT 5\'-11" WGT 205 lb FEDERAL LIMITS APPLY ISS 10/14/2021',
 'output': '\n9939 VAN RUITEN ST',
 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nExtract address from the following input:\nCalifornia DRIVER LICENSE 081202 LN SOTO MUNGUIA FN ALEC PIERRE 9939 VAN RUITEN ST BELLFLOWER, CA 90706 SEX M AM DONOR USA DLY7679862 EXP 08/12/2025 DOB 08/12/2002 AGE 2 IN 7023 DD 08/04/2021606A3/E5FD/25 CLASS C END NONE RSTR NONE 08122002 HAIR BRN EYES HZL HGT 5\'-11" WGT 205 lb FEDERAL LIMITS APPLY ISS 10/14/2021\n\n### Response:\n9939 VAN RUITEN ST\n'}